# 1 citation per year

In [ ]:
cp = pd.read_excel('./citation-report.xls')
cp['GCS'] = cp[[2007+i for i in range(10)]].sum(axis=1)
cp['CitedPerYear'] = cp['GCS']/(2017-cp['Publication Year'])
cp.sort_values(by='CitedPerYear', ascending=False, inplace=True)
cp['Rank'] = range(1,cp.shape[0]+1)
cp.reset_index(inplace=True)
cols = ['Rank','Title','Authors','Source Title','Publication Year','GCS', *[2007+i for i in range(10)], 'CitedPerYear']
final = cp[cols].rename(columns={'Source Title': 'Journal'})
final.to_csv('./cited-forward.csv', index=False)

# 2 burst detection

### 2.1 extract keywords

In [179]:
import re
with open('./total.bib') as f:
    lines = f.readlines()
keywordsByYear = {}
still = False
for line in lines:
    if line[0] == '@':
        still = False
        
    if still and line[0] == ' ':
        line = line.strip()

        if line[-1] == ',':
            still = False
            kw += line.strip()[:-3].split('; ')
            try:
                keywordsByYear[year].extend(kw)
            except:
                keywordsByYear[year] = kw.copy()
        else:
            kw += line.split('; ')
        
        
    line = line.strip()
    if line[:4] == 'Year':
        year = re.search(r'{{(\d+)}}', line).group(1)
    
    if line[:10] == 'Keywords =':
        if line[-1] == ',':
            kw = line[13:-3].strip().split('; ')
            try:
                keywordsByYear[year].extend(kw)
            except:
                keywordsByYear[year] = kw.copy()
        else:
            kw = line[13:].strip().split('; ') 
            still = True
    

### 2.2 stem and count by year

In [293]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from collections import Counter
import json
   
ps = PorterStemmer() 
stemKeywordsByYear = {}
top20 = {}
for year, l in keywordsByYear.items():
    counter = Counter() 
    for s in l:
        s = s.replace('(', '').replace(')', '').replace(';', '').replace(':','').replace('and', '').replace('of', '')
        words = word_tokenize(s) 
        counter.update([ps.stem(w) for w in words])
    stemKeywordsByYear[year] = dict(counter)
    top20[year] = counter.most_common(20)
with open('./stem-keywords-by-year.json', 'w') as f:
    json.dump(stemKeywordsByYear, f)

### 2.3 top 20 keywords(stemed) every year

In [295]:
top20_df = pd.DataFrame(top20).applymap(lambda x: '%s,%d'%x)
top20_df.to_csv('keywords-freq-top20.csv', index=False)
top20_df

,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,"manufactur,78","manufactur,68","manufactur,42","manufactur,33","manufactur,34","manufactur,24","manufactur,22","manufactur,26","manufactur,33","manufactur,15",...,"manufactur,23","manufactur,14","system,14","system,24","manufactur,20","manufactur,11","system,14","system,26","intellig,3","intellig,10"
1,"system,61","system,53","system,26","system,16","system,18","system,16","system,13","intellig,19","system,27","system,11",...,"system,14","system,13","manufactur,12","manufactur,20","system,16","system,8","manufactur,11","manufactur,26","manufactur,2","system,10"
2,"smart,43","smart,32","intellig,22","intellig,11","intellig,10","intellig,15","real-tim,12","system,18","intellig,16","control,10",...,"intellig,12","intellig,8","intellig,9","control,13","intellig,11","intellig,8","control,7","intellig,22","neural,2","manufactur,9"
3,"product,35","intellig,28","process,18","process,8","real-tim,7","simul,9","intellig,7","schedul,7","control,10","process,10",...,"control,10","control,6","control,7","intellig,12","control,9","control,4","intellig,5","control,9","network,2","control,5"
4,"factori,34","process,17","smart,17","plan,7","monitor,7","control,6","rfid,6","model,6","flexibl,7","intellig,6",...,"schedul,5","model,5","real-tim,5","network,9","network,7","flexibl,3","flexibl,4","artifici,8","fuzzi,2","cell,3"
5,"intellig,26","control,16","factori,11","real-tim,6","process,7","process,5","network,5","simul,6","product,6","real-tim,5",...,"simul,5","simul,5","agent,4","model,7","schedul,6","neural,3","schedul,4","network,8","logic,2","integr,3"
6,"industri,23","product,15","machin,9","schedul,6","dynam,6","time,4","agent,5","control,5","comput,6","model,5",...,"model,5","smart,5","respons,3","fuzzi,7","neural,6","network,3","network,3","neural,7","object-ori,2","engin,2"
7,"process,23","model,14","control,9","product,6","manag,6","product,4","control,5","real-tim,5","wireless,5","machin,5",...,"flexibl,5","comput,5","manag,3","neural,5","plan,5","the,3","design,3","fuzzi,6","configur,2","expert,2"
8,"model,15","design,12","product,7","manag,6","design,5","real-tim,4","enterpris,4","agent,4","schedul,5","design,4",...,"product,4","real-tim,4","time,3","process,4","agent,4","simul,2","simul,3","agent,5","autom,2","robot,2"
9,"energi,14","factori,12","manag,7","machin,5","enterpris,4","design,4","wireless,4","distribut,4","smart,5","qualiti,4",...,"sensor,4","schedul,4","schedul,2","agent,4","distribut,4","autom,2","neural,2","expert,4","system,2","plan,2"


### 2.4 keyword frequency selected by author

In [115]:
from burst_detection import burst_detection
from burst_detection import enumerate_bursts

In [187]:
selectWords = ['expert', 'flexibl', 'network', 'neural', 'artifici', 'fuzzi', 'system', 'plan', 'distribut', 'function', 
               'model', 'base', 'wireless', 'rfid', 'optim', 'detect', 'monitor', 'cloud', 'thing', 'smart']

In [296]:
selectStat = {w:{} for w in selectWords}
for year,v in stemKeywordsByYear.items():
    for w in selectWords:
        selectStat[w][year] = v[w] if w in v else 0
selectStat = pd.DataFrame(selectStat)
selectStat['sum'] = selectStat.sum(axis=1)
selectStat.index.name = 'Year'
selectStat.to_csv('./selected-keywords-freq.csv')
selectStat

,expert,flexibl,network,neural,artifici,fuzzi,system,plan,distribut,function,...,base,wireless,rfid,optim,detect,monitor,cloud,thing,smart,sum
Year,,,,,,,,,,,,,,,,,,,,,
1995,2,1,1,1,1,0,10,2,1,0,...,0,0,0,0,0,0,0,0,1,20
1996,1,1,2,2,1,2,2,0,0,0,...,0,0,0,1,0,0,0,0,0,12
1997,4,3,8,7,8,6,26,2,2,0,...,0,0,0,1,0,3,0,0,0,71
1998,2,4,3,2,2,1,14,0,0,0,...,0,0,0,1,0,0,0,0,0,31
1999,1,3,3,3,0,0,8,1,0,0,...,0,0,1,0,0,0,0,0,2,23
2000,1,0,7,6,1,2,16,5,4,4,...,0,0,1,0,0,2,0,0,1,52
2001,1,4,9,5,0,7,24,3,3,0,...,3,0,0,1,0,1,0,0,2,70
2002,0,2,1,1,1,1,14,0,2,0,...,0,0,0,0,0,0,0,0,1,23
2003,0,1,3,2,1,1,13,1,2,2,...,0,0,0,1,0,1,0,0,5,38


### 2.5 burst

In [233]:
for word in selectWords:
    q, *_ = burst_detection(selectStat[word], selectStat['sum'], 22, s=4, gamma=0.2, smooth_win=3)
    res = enumerate_bursts(q,word)
    res.begin += 1995
    res.end += 1995
    print(word)
    print(res,'\n\n')

expert
    label  begin   end weight
0  expert   1996  2000    NaN 


flexibl
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


network
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


neural
    label  begin   end weight
0  neural   1997  2001    NaN 


artifici
      label  begin   end weight
0  artifici   1996  1998    NaN 


fuzzi
   label  begin   end weight
0  fuzzi   1997  1997    NaN 


system
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


plan
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


distribut
       label  begin   end weight
0  distribut   2001  2002    NaN
1  distribut   2008  2010    NaN 


function
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


model
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


base
Empty DataFrame
Columns: [label, begin, end, weight]
Index: [] 


wireless
      label  begin   end weight
0  wireless   2008  2011    NaN 


rfid
  label  begin